<h1>Global Impact - Grupo Sigma</h1>
Modelo escolhido: KNN <br>
Justificativa: dado a natureza dos dados, o tamanho do dataset e a saída esperada, a escolha de um algoritmo de aprendizado supervizionado como o KNN se mostra pertinente.



In [23]:
#Importando bibliotecas que serão utilizadas
import math
import pandas as pd

<h3> Declarando as funções </h3>


In [24]:
def distancia_euclidiana (p1,p2):
  dimensao = len(p1)
  soma = 0
  for i in range(dimensao):
    soma += (p1[i] - p2[i])**2
  return math.sqrt(soma)
def info_dataset(amostras, info= True):
  output1 = 0
  output2 = 0
  for amostra in amostras.iloc[:, -1]:
    if amostra == 1:
      output1 += 1
    else:
      output2 += 1
  if info == True:
    print(f'Total de amostras: {len(amostras)}')
    print(f'Total normal: {output1}')
    print(f'Total alterado: {output2}')
  return [len(amostras), output1, output2]

def knn(treinamento, nova_amostra, k):
  distancias = {}
  tamanho_treino = len(treinamento)
  for i in range (tamanho_treino):
    d= distancia_euclidiana(treinamento[i], nova_amostra)
    distancias[i] = d 
  k_vizinhos = sorted(distancias, key = distancias.get)[:k]

  qtd_output1 = 0
  qtd_output2 = 0
  for indice in k_vizinhos:
    if treinamento[indice][-1] == 1:
      qtd_output1 += 1
    else:
      qtd_output2 += 1
  if qtd_output1 > qtd_output2:
    return 1
  else:
    return 0
def min_max_scaling(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        
    return df_norm


<h3>Lendo CSV e transformando em um dataframe</h3>



In [25]:
df = pd.read_csv("dataset_tomate.csv")
df

,id,trat,NDVI_d28,SAVI_d28,GNDVI_d28,MCARI1_d28,SR_d28,NDVI_d01,SAVI_d01,GNDVI_d01,MCARI1_d01,SR_d01,NDVI_d04,SAVI_d04,GNDVI_d04,MCARI1_d04,SR_d04,NDVI_d08,SAVI_d08,GNDVI_d08,MCARI1_d08,SR_d08,Classe
0,1,102,0.798284,1.197380,0.692652,16710.29721,9.728142,0.729767,1.094620,0.698726,21499.94698,6.907142,0.713179,1.069718,0.662425,13090.11459,6.216968,0.684125,1.026146,0.636865,13352.81776,5.421333,1
1,2,11,0.806955,1.210392,0.705323,19513.85630,10.557411,0.774359,1.161514,0.717367,30041.91766,8.531835,0.777758,1.166604,0.680269,22731.09954,8.380453,0.725521,1.088240,0.669113,16190.82038,6.491860,0
2,3,103,0.825104,1.237610,0.708468,18074.06986,11.141408,0.786941,1.180385,0.730471,29312.17626,8.982521,0.730708,1.096008,0.680238,12330.21702,6.670915,0.728171,1.092214,0.655206,15313.16198,6.551268,1
3,4,112,0.788884,1.183279,0.664787,16226.23017,8.990752,0.743874,1.115782,0.690141,22610.47885,7.109600,0.707999,1.061948,0.644786,12085.07787,6.017714,0.728161,1.092201,0.640739,16562.17418,6.608121,1
4,5,114,0.764986,1.147432,0.673490,14920.80711,8.269806,0.715948,1.073894,0.695619,22002.08151,6.447621,0.688155,1.032176,0.660206,10053.17885,5.571702,0.694320,1.041437,0.642373,13760.95583,5.665567,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,128,82,0.750210,1.125268,0.644542,14328.38849,7.591731,0.707780,1.061642,0.668688,21656.64142,6.144368,0.683641,1.025412,0.626456,11786.65046,5.459075,0.658993,0.988445,0.617495,11584.92863,4.928687,1
128,129,81,0.807576,1.211321,0.706454,18155.87571,10.348776,0.774815,1.162196,0.727687,27744.20116,8.538699,0.734909,1.102315,0.682965,14698.32604,6.903509,0.710542,1.065769,0.663861,13727.74708,6.049484,1
129,130,84,0.781299,1.171903,0.671280,16684.03811,9.000704,0.734850,1.102248,0.691017,25430.27568,7.154168,0.708276,1.062361,0.654872,12633.29162,6.177030,0.672909,1.009326,0.626687,13901.73759,5.240016,1
130,131,92,0.760722,1.141037,0.660274,14967.20998,8.141685,0.729611,1.094388,0.687685,23396.21203,6.989578,0.712179,1.068222,0.644261,14209.13957,6.259536,0.685162,1.027701,0.627243,14067.88995,5.514438,1


<h3>Normalização dos dados</h3>

In [26]:
 df = df.drop(['id' ], axis=1)
 df

,trat,NDVI_d28,SAVI_d28,GNDVI_d28,MCARI1_d28,SR_d28,NDVI_d01,SAVI_d01,GNDVI_d01,MCARI1_d01,SR_d01,NDVI_d04,SAVI_d04,GNDVI_d04,MCARI1_d04,SR_d04,NDVI_d08,SAVI_d08,GNDVI_d08,MCARI1_d08,SR_d08,Classe
0,102,0.798284,1.197380,0.692652,16710.29721,9.728142,0.729767,1.094620,0.698726,21499.94698,6.907142,0.713179,1.069718,0.662425,13090.11459,6.216968,0.684125,1.026146,0.636865,13352.81776,5.421333,1
1,11,0.806955,1.210392,0.705323,19513.85630,10.557411,0.774359,1.161514,0.717367,30041.91766,8.531835,0.777758,1.166604,0.680269,22731.09954,8.380453,0.725521,1.088240,0.669113,16190.82038,6.491860,0
2,103,0.825104,1.237610,0.708468,18074.06986,11.141408,0.786941,1.180385,0.730471,29312.17626,8.982521,0.730708,1.096008,0.680238,12330.21702,6.670915,0.728171,1.092214,0.655206,15313.16198,6.551268,1
3,112,0.788884,1.183279,0.664787,16226.23017,8.990752,0.743874,1.115782,0.690141,22610.47885,7.109600,0.707999,1.061948,0.644786,12085.07787,6.017714,0.728161,1.092201,0.640739,16562.17418,6.608121,1
4,114,0.764986,1.147432,0.673490,14920.80711,8.269806,0.715948,1.073894,0.695619,22002.08151,6.447621,0.688155,1.032176,0.660206,10053.17885,5.571702,0.694320,1.041437,0.642373,13760.95583,5.665567,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,82,0.750210,1.125268,0.644542,14328.38849,7.591731,0.707780,1.061642,0.668688,21656.64142,6.144368,0.683641,1.025412,0.626456,11786.65046,5.459075,0.658993,0.988445,0.617495,11584.92863,4.928687,1
128,81,0.807576,1.211321,0.706454,18155.87571,10.348776,0.774815,1.162196,0.727687,27744.20116,8.538699,0.734909,1.102315,0.682965,14698.32604,6.903509,0.710542,1.065769,0.663861,13727.74708,6.049484,1
129,84,0.781299,1.171903,0.671280,16684.03811,9.000704,0.734850,1.102248,0.691017,25430.27568,7.154168,0.708276,1.062361,0.654872,12633.29162,6.177030,0.672909,1.009326,0.626687,13901.73759,5.240016,1
130,92,0.760722,1.141037,0.660274,14967.20998,8.141685,0.729611,1.094388,0.687685,23396.21203,6.989578,0.712179,1.068222,0.644261,14209.13957,6.259536,0.685162,1.027701,0.627243,14067.88995,5.514438,1


In [27]:
df = min_max_scaling(df)
df

,trat,NDVI_d28,SAVI_d28,GNDVI_d28,MCARI1_d28,SR_d28,NDVI_d01,SAVI_d01,GNDVI_d01,MCARI1_d01,SR_d01,NDVI_d04,SAVI_d04,GNDVI_d04,MCARI1_d04,SR_d04,NDVI_d08,SAVI_d08,GNDVI_d08,MCARI1_d08,SR_d08,Classe
0,0.281734,0.834531,0.834523,0.834309,0.665776,0.704641,0.558394,0.558370,0.728023,0.182397,0.446168,0.475771,0.475713,0.672413,0.262140,0.365521,0.175348,0.175323,0.372830,0.159900,0.094783,1.0
1,0.000000,0.883027,0.883043,0.912197,0.992956,0.822972,0.823747,0.823741,0.855947,0.643688,0.749900,0.913244,0.913257,0.781408,0.952983,0.871357,0.433084,0.433059,0.594257,0.400143,0.290298,0.0
2,0.284830,0.984542,0.984539,0.931529,0.824931,0.906304,0.898617,0.898605,0.945877,0.604280,0.834155,0.594514,0.594438,0.781218,0.207688,0.471656,0.449585,0.449557,0.498770,0.325847,0.301148,1.0
3,0.312693,0.781951,0.781940,0.663029,0.609285,0.599421,0.642341,0.642321,0.669106,0.242369,0.484017,0.440683,0.440622,0.564671,0.190122,0.318934,0.449521,0.449501,0.399433,0.431579,0.311532,1.0
4,0.318885,0.648280,0.648266,0.716522,0.456940,0.496548,0.476164,0.476149,0.706698,0.209514,0.360261,0.306251,0.306168,0.658864,0.044522,0.214654,0.238820,0.238790,0.410649,0.194450,0.139388,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,0.219814,0.565634,0.565617,0.538587,0.387804,0.399791,0.427559,0.427545,0.521879,0.190859,0.303569,0.275675,0.275623,0.452709,0.168737,0.188321,0.018871,0.018831,0.239827,0.010244,0.004808,1.0
128,0.216718,0.886502,0.886506,0.919147,0.834478,0.793201,0.826462,0.826449,0.926773,0.519605,0.751183,0.622974,0.622923,0.797878,0.377379,0.526038,0.339826,0.339788,0.558200,0.191638,0.209505,1.0
129,0.226006,0.739524,0.739520,0.702940,0.662712,0.600841,0.588642,0.588631,0.675117,0.394646,0.492349,0.442555,0.442486,0.626283,0.229405,0.356183,0.105515,0.105504,0.302947,0.206367,0.061668,1.0
130,0.250774,0.624432,0.624419,0.635290,0.462355,0.478266,0.557466,0.557449,0.652250,0.284801,0.461579,0.468996,0.468954,0.561466,0.342326,0.375474,0.181801,0.181776,0.306765,0.220432,0.111787,1.0


<h3>Definindo conjunto de treinamento e teste</h3>

In [28]:
porcentagem = 0.7
treinamento = []
teste = []

_, output1, output2 = info_dataset(df, info=True)

max_output1 = int(porcentagem*output1)
max_output2 = int(porcentagem*output2)

total_output1 = 0
total_output2 = 0


Total de amostras: 132
Total normal: 91
Total alterado: 41


In [29]:
amostras = df.values.tolist()
for amostra in amostras:
  if (total_output1 + total_output2) < (max_output1 + max_output2):
    treinamento.append(amostra)
    if amostra[-1] == 1 and total_output1 < max_output1:
        total_output1 += 1
    else:
      total_output2 += 1
  else: 
    #senao, o conjunto de treinamento ja foi preenchido
    teste.append(amostra)

<h3>Treinando o modelo e realizando o teste</h3>

In [30]:
acertos = 0
k = 11

for amostra in teste:
  classe = knn (treinamento, amostra, k)
  if amostra[-1] == classe: 
    acertos += 1
print(f'Total de treinamento {len(treinamento)}')
print(f'Total de testes {len(teste)}')
print(f'Total de acertos {acertos}')
print(f'Porcentagem de acerto {100*acertos/len(teste)}')

Total de treinamento 91
Total de testes 41
Total de acertos 36
Porcentagem de acerto 87.8048780487805
